In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
model = Ollama(model="mistral")

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "soccer"})

" Why don't grasshoppers watch soccer?\n\nBecause they hate penalties! (Get it? Because grasshoppers hop and a penalty in soccer is a foul that results in a free kick from the penalty spot.)"

# 基础

在 `langchain`中最简单的使用就是：prompt + model + output parser，类似上面的例子。在代码中我们可以看到类似： `chain = prompt | model | output_parser`. 这里的 `|` 类似unix中的管道操作符，将chain的不同组件串接在一起。

## prompt

在 `langchain` 中`prompt`是 `BasePromptTemplate` 的实现类，接收模板参数词典，产出`PromptValue`。`PromptValue`是一个完整prompt的封装，可以直接传递给 `LLM`(接收字符串输入)或者 `ChatModel` (接收消息序列)

## model

`PropmtValue`会被传给模型（`model`），对于上面的例子是一个 `ChatModel`, 这个类型的模型会输出 `BaseMessage`, `LLM`类型则是会返回字符串。

## Output Parser

在代码的最后，我们将模型`model`的输出传递到 `output_parser`, 这是一个 `BaseOutputParser`类型的对象，将字符串或者`BaseMessage`作为输入。例如 `StrOutputParser`是将输入转换为字符串

In [7]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [4]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [8]:
message = model.invoke(prompt_value)
message

'\nSure, here\'s a short joke about ice cream:\n\nWhy did the ice cream cone go to the party?\n\nBecause it was feeling "scoopy"!'

# RAG

RAG表示 retrieval-augmented generation

In [5]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser
chain.invoke("where did harrison work?")

' Harrison worked at Kensho, based on the provided context. However, the first document about bears and honey is not relevant to the answer of this question.'